# **Chapter 13: Identity and Access Management (IAM) Deep Dive**

## Introduction: Identity as the New Security Perimeter

The network perimeter has dissolved. With applications distributed across multi-cloud environments, remote work normalized, and data flowing between on-premises data centers and serverless functions, traditional network-based security controls—firewalls, VPNs, and network segmentation—are necessary but insufficient. In this landscape, identity becomes the primary security control plane: who can access which resources, under what conditions, and with what privileges defines the security posture of modern cloud architectures.

Yet IAM remains the most misconfigured and exploited layer of cloud security. Overly permissive policies, long-lived credentials, and stale access permissions create pathways for lateral movement, privilege escalation, and data exfiltration. The 2023 Verizon Data Breach Investigations Report identified compromised credentials as the initial attack vector in 49% of breaches, with cloud misconfiguration—predominantly IAM—contributing to the majority of cloud-related incidents.

This chapter moves beyond basic IAM concepts to explore sophisticated access control architectures. We will implement Attribute-Based Access Control (ABAC) for dynamic authorization, secure non-human identities through workload identity federation, establish Just-in-Time (JIT) access for privileged operations, and design cross-account access patterns that maintain security boundaries while enabling operational efficiency. These patterns provide the identity foundation for the infrastructure security controls explored in the next chapter.

---

## 13.1 IAM Architecture Fundamentals

Effective IAM architectures distinguish between authentication (verifying identity) and authorization (determining permissions), applying the principle of least privilege at every layer while maintaining auditability of all access decisions.

### 13.1.1 The Identity Hierarchy

**Users:** Human identities (employees, contractors) requiring console and CLI access. Best practice: Eliminate long-term access keys; use temporary credentials via SSO.

**Groups:** Collections of users sharing common permissions (e.g., `DataEngineers`, `SecurityAnalysts`). Groups cannot be principals in policies but simplify permission assignment.

**Roles:** IAM entities with permission policies that can be assumed by users, applications, or services. Roles have no permanent credentials; they return temporary access tokens upon assumption.

**Policies:** JSON documents defining permissions (Allow/Deny) attached to identities or resources. Policies evaluate explicitly—everything is denied by default unless explicitly allowed, and explicit denies override explicit allows.

**AWS IAM Policy Structure Deep Dive:**

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "ExplicitAllowWithConditions",
      "Effect": "Allow",
      "Principal": {
        "AWS": "arn:aws:iam::123456789012:role/DataProcessorRole"
      },
      "Action": [
        "s3:GetObject",
        "s3:PutObject"
      ],
      "Resource": [
        "arn:aws:s3:::customer-data-prod",
        "arn:aws:s3:::customer-data-prod/${aws:userid}/*"
      ],
      "Condition": {
        "StringEquals": {
          "s3:x-amz-server-side-encryption": "aws:kms",
          "aws:RequestedRegion": ["us-east-1", "eu-west-1"]
        },
        "IpAddress": {
          "aws:SourceIp": ["10.0.0.0/8", "192.168.1.0/24"]
        },
        "Bool": {
          "aws:MultiFactorAuthPresent": "true",
          "aws:SecureTransport": "true"
        },
        "DateGreaterThan": {
          "aws:CurrentTime": "2026-01-01T00:00:00Z"
        },
        "ArnLike": {
          "aws:PrincipalTag/Department": "Engineering"
        }
      }
    },
    {
      "Sid": "ExplicitDenyWildCard",
      "Effect": "Deny",
      "Action": "s3:*",
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "s3:prefix": ["*", ""]  // Prevents listing all buckets
        }
      }
    }
  ]
}
```

**Critical Implementation Details:**
- **Variable Substitution:** `${aws:userid}` creates user-specific namespaces within buckets, preventing cross-user data access
- **Condition Operators:** `StringEquals` for exact matches, `ArnLike` for wildcard ARN matching, `Null` for checking attribute existence
- **Secure Transport:** `aws:SecureTransport` enforces TLS; without this, HTTP requests would be allowed if other conditions pass
- **Principal Tags:** ABAC foundation—permissions dynamically adjust based on the principal's organizational attributes

### 13.1.2 RBAC vs. ABAC: Evolution of Access Control

**Role-Based Access Control (RBAC):**
Permissions assigned based on job function (e.g., "Database Administrator" role gets RDS permissions). Simple to audit but rigid—requires new roles for edge cases and creates role proliferation.

**Attribute-Based Access Control (ABAC):**
Permissions determined dynamically by attributes of the principal (who), resource (what), and environment (context). Enables fine-grained access without policy proliferation.

**ABAC Implementation Example:**

```json
// IAM Policy allowing engineers to access resources tagged with their project
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "ec2:StartInstances",
        "ec2:StopInstances",
        "ec2:RebootInstances"
      ],
      "Resource": "arn:aws:ec2:us-east-1:123456789012:instance/*",
      "Condition": {
        "StringEquals": {
          "ec2:ResourceTag/Project": "${aws:PrincipalTag/Project}",
          "ec2:ResourceTag/Environment": "${aws:PrincipalTag/Environment}"
        },
        "ForAllValues:StringEquals": {
          "aws:TagKeys": ["Project", "Environment", "Owner"]
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": "ec2:CreateTags",
      "Resource": "arn:aws:ec2:us-east-1:123456789012:instance/*",
      "Condition": {
        "StringEquals": {
          "ec2:CreateAction": "RunInstances"
        }
      }
    }
  ]
}
```

**ABAC Benefits:**
- **Scalability:** New engineers automatically get appropriate access by tagging their IAM user/role with `Project=Alpha`, without modifying policies
- **Dynamic Authorization:** If an engineer moves to Project Beta, changing their principal tag immediately shifts permissions—no policy updates required
- **Resource Isolation:** Engineers cannot accidentally (or maliciously) interact with Project Gamma resources because the tags won't match

**Azure ABAC with Conditions:**

```json
{
  "properties": {
    "roleName": "Storage Blob Data Contributor with Conditions",
    "description": "Allows read/write access to blobs with specific tags",
    "type": "customRole",
    "assignableScopes": ["/subscriptions/{subscription-id}"],
    "permissions": [
      {
        "actions": ["Microsoft.Storage/storageAccounts/blobServices/containers/blobs/*"],
        "notActions": [],
        "dataActions": [
          "Microsoft.Storage/storageAccounts/blobServices/containers/blobs/read",
          "Microsoft.Storage/storageAccounts/blobServices/containers/blobs/write"
        ],
        "notDataActions": [],
        "condition": {
          "version": "2.0",
          "description": "Access only to blobs with matching project tag",
          "scope": "/subscriptions/{subscription-id}/resourceGroups/{resource-group}",
          "if": {
            "allOf": [
              {
                "field": "Microsoft.Storage/storageAccounts/blobServices/containers/blobs/tags[Project]",
                "equals": "[requestContext().user.claims['project']]"
              },
              {
                "field": "Microsoft.Storage/storageAccounts/blobServices/containers/blobs/tags[Classification]",
                "in": ["Internal", "Public"]
              }
            ]
          }
        }
      }
    ]
  }
}
```

---

## 13.2 Service Accounts and Workload Identity

Non-human identities (applications, services, CI/CD pipelines) outnumber human users in cloud environments by orders of magnitude. Traditional service account keys (long-lived passwords/credentials) create significant risk—if leaked, they provide indefinite access until manually rotated.

### 13.2.1 Eliminating Long-Term Credentials

**The Anti-Pattern:**
```bash
# DANGEROUS: Long-lived service account key
gcloud iam service-accounts keys create key.json --iam-account=app@project.iam.gserviceaccount.com
# This key is valid until explicitly deleted, potentially years
```

**The Solution: Workload Identity Federation**

Workload Identity Federation allows external identities (GitHub Actions, GitLab CI, on-premises Kubernetes, Azure AD) to impersonate cloud service accounts without long-term keys, using short-lived OIDC tokens.

**AWS: IAM Roles for Service Accounts (IRSA) on EKS:**

```yaml
# Kubernetes ServiceAccount mapped to IAM Role
apiVersion: v1
kind: ServiceAccount
metadata:
  name: payment-processor
  namespace: production
  annotations:
    # Trust relationship established via OIDC provider
    eks.amazonaws.com/role-arn: arn:aws:iam::123456789012:role/PaymentProcessorRole
    eks.amazonaws.com/sts-regional-endpoints: "true"
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: payment-service
spec:
  template:
    spec:
      serviceAccountName: payment-processor  # Pod assumes IAM Role automatically
      containers:
        - name: app
          image: payment-service:1.2.3
          env:
            - name: AWS_REGION
              value: "us-east-1"
            # No AWS_ACCESS_KEY_ID or AWS_SECRET_ACCESS_KEY needed
```

**Trust Policy (IAM Role):**
```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Federated": "arn:aws:iam::123456789012:oidc-provider/oidc.eks.us-east-1.amazonaws.com/id/EXAMPLED539D4633E53DE1B716D3041E"
      },
      "Action": "sts:AssumeRoleWithWebIdentity",
      "Condition": {
        "StringEquals": {
          "oidc.eks.us-east-1.amazonaws.com/id/EXAMPLED539D4633E53DE1B716D3041E:sub": "system:serviceaccount:production:payment-processor",
          "oidc.eks.us-east-1.amazonaws.com/id/EXAMPLED539D4633E53DE1B716D3041E:aud": "sts.amazonaws.com"
        }
      }
    }
  ]
}
```

**Security Benefits:**
- **No Static Keys:** The pod receives temporary credentials (valid ~1 hour) via the EKS OIDC provider
- **Audience Binding:** The `aud` condition ensures tokens are only valid for AWS STS, preventing replay to other OIDC consumers
- **Namespace Isolation:** Only pods running as `payment-processor` service account in `production` namespace can assume the role

**GCP Workload Identity Federation (GitHub Actions to GCP):**

```yaml
# GitHub Actions workflow using Workload Identity Federation
name: Deploy to GCP
on:
  push:
    branches: [main]

jobs:
  deploy:
    permissions:
      contents: 'read'
      id-token: 'write'  # Required for OIDC token generation
    
    steps:
      - uses: actions/checkout@v4
      
      - id: auth
        uses: google-github-actions/auth@v2
        with:
          workload_identity_provider: 'projects/123456789/locations/global/workloadIdentityPools/github-pool/providers/github-provider'
          service_account: 'deployer@my-project.iam.gserviceaccount.com'
          # No JSON key file needed - uses OIDC token from GitHub
      
      - name: Deploy to Cloud Run
        uses: google-github-actions/deploy-cloudrun@v2
        with:
          service: my-service
          source: ./
```

**Terraform Configuration for Workload Identity Pool:**

```hcl
resource "google_iam_workload_identity_pool" "github_pool" {
  workload_identity_pool_id = "github-pool"
  display_name              = "GitHub Actions Pool"
  description               = "Identity pool for GitHub Actions CI/CD"
}

resource "google_iam_workload_identity_pool_provider" "github_provider" {
  workload_identity_pool_id          = google_iam_workload_identity_pool.github_pool.workload_identity_pool_id
  workload_identity_pool_provider_id = "github-provider"
  
  attribute_mapping = {
    "google.subject"       = "assertion.sub"  # Maps GitHub org/repo to Google identity
    "attribute.repository" = "assertion.repository"
    "attribute.actor"      = "assertion.actor"
  }
  
  attribute_condition = "assertion.repository.startsWith('myorg/')"  # Only trusted repos
  
  oidc {
    issuer_uri = "https://token.actions.githubusercontent.com"
  }
}

# Allow specific repository to impersonate service account
resource "google_service_account_iam_member" "workload_identity_user" {
  service_account_id = google_service_account.deployer.name
  role               = "roles/iam.workloadIdentityUser"
  member             = "principalSet://iam.googleapis.com/${google_iam_workload_identity_pool.github_pool.name}/attribute.repository/myorg/my-repo"
}
```

### 13.2.2 Credential Rotation and Expiration

Even temporary credentials require rotation strategies. AWS STS credentials expire (default 1 hour, max 12 hours), but service account keys in GCP or storage account keys in Azure persist until deleted.

**Automated Rotation with AWS Secrets Manager:**

```python
import boto3
import json
from botocore.exceptions import ClientError

def rotate_service_account_key(secret_id):
    """
    Lambda function triggered by Secrets Manager rotation schedule
    Implements the rotation strategy for OAuth2 client credentials
    """
    secrets_client = boto3.client('secretsmanager')
    
    # Rotation phases: create new version, distribute, test, deprecate old
    try:
        # Get current secret metadata
        metadata = secrets_client.describe_secret(SecretId=secret_id)
        
        # Generate new OAuth2 credentials (hypothetical external API)
        new_credentials = generate_new_oauth_credentials()
        
        # Stage new version (AWSPENDING)
        secrets_client.put_secret_value(
            SecretId=secret_id,
            ClientRequestToken=rotation_token,
            SecretString=json.dumps(new_credentials),
            VersionStages=['AWSPENDING']
        )
        
        # Update all consuming applications (via SSM Parameter Store or EventBridge)
        notify_consumers_new_version_available(secret_id, rotation_token)
        
        # Validate new credentials work
        if validate_credentials(new_credentials):
            # Move new version to AWSCURRENT
            secrets_client.update_secret_version_stage(
                SecretId=secret_id,
                VersionStage='AWSCURRENT',
                MoveToVersionId=rotation_token,
                RemoveFromVersionId=metadata['VersionIdsToStages']['AWSCURRENT']
            )
            
            # Schedule old version deletion (7-day grace period)
            secrets_client.update_secret_version_stage(
                SecretId=secret_id,
                VersionStage='AWSPREVIOUS',
                MoveToVersionId=metadata['VersionIdsToStages']['AWSCURRENT'][0]
            )
            
    except ClientError as e:
        logger.error(f"Rotation failed: {e}")
        raise
```

---

## 13.3 Just-in-Time (JIT) Access and Privileged Identity Management

Standing access (permanent privileges) violates least privilege principles. Users accumulate permissions over time, creating "permission creep" and expanding the attack surface. Just-in-Time (JIT) access grants elevated privileges only when needed, for limited duration, with approval workflows.

### 13.3.1 JIT Architecture Components

**Request Flow:**
1. Engineer requests elevation via CLI or portal (e.g., "Need admin access to production database for 2 hours")
2. Approval workflow routes to manager or automated policy check
3. Upon approval, temporary role assumption granted (e.g., AWS STS with 2-hour expiration)
4. Session recorded and monitored
5. Access automatically revoked after time limit or task completion

**Implementation with AWS IAM Identity Center (formerly SSO) and Lambda:**

```python
# AWS Lambda for JIT access orchestration
import boto3
import json
import uuid
from datetime import datetime, timedelta

def request_elevation(event, context):
    """
    API Gateway handler for JIT access requests
    """
    request_body = json.loads(event['body'])
    user_id = event['requestContext']['identity']['user']
    requested_role = request_body['role_arn']  # e.g., arn:aws:iam::PROD:role/DatabaseAdmin
    duration = min(request_body['duration_hours'], 4)  # Cap at 4 hours
    justification = request_body['justification']
    
    # Check if user has base permissions to request this elevation
    if not can_request_role(user_id, requested_role):
        return {'statusCode': 403, 'body': 'Unauthorized to request this role'}
    
    # Create approval ticket in DynamoDB
    ticket_id = str(uuid.uuid4())
    ticket = {
        'ticket_id': ticket_id,
        'requester': user_id,
        'requested_role': requested_role,
        'duration_hours': duration,
        'justification': justification,
        'status': 'PENDING_APPROVAL',
        'created_at': datetime.utcnow().isoformat(),
        'expires_at': (datetime.utcnow() + timedelta(hours=24)).isoformat()
    }
    
    # Store in DynamoDB
    table = boto3.resource('dynamodb').Table('jit-access-requests')
    table.put_item(Item=ticket)
    
    # Notify approvers via SNS
    sns = boto3.client('sns')
    sns.publish(
        TopicArn='arn:aws:sns:us-east-1:123456789012:jit-approval-requests',
        Subject=f'JIT Access Request: {user_id}',
        Message=json.dumps(ticket)
    )
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'ticket_id': ticket_id,
            'status': 'PENDING_APPROVAL',
            'message': 'Request submitted for approval'
        })
    }

def approve_elevation(event, context):
    """
    Triggered by approver action (via API or Slack integration)
    """
    approval_data = json.loads(event['body'])
    ticket_id = approval_data['ticket_id']
    approver_id = event['requestContext']['identity']['user']
    
    # Retrieve ticket
    table = boto3.resource('dynamodb').Table('jit-access-requests')
    ticket = table.get_item(Key={'ticket_id': ticket_id})['Item']
    
    # Validate approver has authority over requester
    if not is_manager_of(approver_id, ticket['requester']):
        return {'statusCode': 403, 'body': 'Not authorized to approve this request'}
    
    # Create temporary permission set in IAM Identity Center
    sso_admin = boto3.client('sso-admin')
    instance_arn = 'arn:aws:sso:::instance/ssoins-1234567890abcdef'
    
    # Create temporary assignment (this is a simplified representation)
    # In production, use AWS IAM Identity Center APIs or custom STS assume role logic
    temporary_credentials = create_temporary_session(
        role_arn=ticket['requested_role'],
        user_id=ticket['requester'],
        duration_hours=ticket['duration_hours'],
        ticket_id=ticket_id
    )
    
    # Update ticket status
    table.update_item(
        Key={'ticket_id': ticket_id},
        UpdateExpression='SET #status = :status, approved_by = :approver, approved_at = :now',
        ExpressionAttributeNames={'#status': 'status'},
        ExpressionAttributeValues={
            ':status': 'APPROVED',
            ':approver': approver_id,
            ':now': datetime.utcnow().isoformat()
        }
    )
    
    # Schedule automatic revocation via EventBridge
    scheduler = boto3.client('scheduler')
    scheduler.create_schedule(
        Name=f'revoke-{ticket_id}',
        ScheduleExpression=f'rate({ticket["duration_hours"]} hours)',
        Target={
            'Arn': 'arn:aws:lambda:us-east-1:123456789012:function:revoke-access',
            'RoleArn': 'arn:aws:iam::123456789012:role/SchedulerRole',
            'Input': json.dumps({'ticket_id': ticket_id})
        },
        FlexibleTimeWindow={'Mode': 'OFF'}
    )
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': 'Access granted',
            'expires_in': f'{ticket["duration_hours"]} hours',
            'temporary_credentials': temporary_credentials
        })
    }
```

### 13.3.2 Break-Glass Procedures

Emergency access (break-glass) accounts bypass normal controls for disaster recovery, but require extreme safeguards:

**Controls:**
- **Physical Hardware Tokens:** MFA not dependent on mobile devices or corporate network
- **Dual-Control:** Requires two authorized individuals to combine credentials
- **Canary Tokens:** Fake credentials that trigger immediate alerts if used (testing for compromise)
- **Automatic Alerting:** Any break-glass usage immediately pages security team
- **Time-Bound:** Credentials valid only during declared emergencies, rotated immediately after use

```hcl
# Terraform: Break-glass account with maximum monitoring
resource "aws_iam_user" "break_glass" {
  name = "break-glass-emergency"
  path = "/emergency/"
  
  tags = {
    Type        = "BreakGlass"
    Monitoring  = "Maximum"
    Rotation    = "PostIncident"
  }
}

resource "aws_iam_user_policy" "break_glass_restrictions" {
  name = "EmergencyAccessOnly"
  user = aws_iam_user.break_glass.name
  
  policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
        Sid    = "AllowCloudTrailAccessForAudit"
        Effect = "Allow"
        Action = [
          "cloudtrail:LookupEvents",
          "cloudtrail:GetTrailStatus"
        ]
        Resource = "*"
      },
      {
        Sid    = "DenyUnlessEmergencyTag"
        Effect = "Deny"
        Action = "*"
        Resource = "*"
        Condition = {
          StringNotEquals = {
            "aws:PrincipalTag/EmergencyMode" = "true"
          }
        }
      }
    ]
  })
}

# EventBridge rule for immediate alerting on break-glass usage
resource "aws_cloudwatch_event_rule" "break_glass_alert" {
  name        = "break-glass-usage"
  description = "Trigger on any break-glass account activity"
  
  event_pattern = jsonencode({
    detail-type = ["AWS API Call via CloudTrail"]
    detail = {
      userIdentity = {
        type = ["IAMUser"]
        userName = ["break-glass-emergency"]
      }
    }
  })
}

resource "aws_cloudwatch_event_target" "sns_alert" {
  rule      = aws_cloudwatch_event_rule.break_glass_alert.name
  target_id = "SecurityTeamAlert"
  arn       = aws_sns_topic.security_alerts.arn
}
```

---

## 13.4 Cross-Account and Cross-Cloud Access Patterns

Multi-cloud architectures require secure authentication and authorization across organizational boundaries without creating identity silos or excessive trust relationships.

### 13.4.1 AWS Cross-Account Role Assumption

The standard pattern for multi-account AWS architectures uses centralized identity accounts assuming roles into workload accounts.

**Trust Relationship with External ID (Third-Party Protection):**

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "AWS": "arn:aws:iam::CENTRAL-ID-ACCOUNT:root"
      },
      "Action": "sts:AssumeRole",
      "Condition": {
        "StringEquals": {
          "sts:ExternalId": "unique-id-provided-by-third-party"
        },
        "IpAddress": {
          "aws:SourceIp": "10.0.0.0/8"
        },
        "Bool": {
          "aws:MultiFactorAuthPresent": "true"
        }
      }
    }
  ]
}
```

**Why External ID Matters:**
When third parties (MSPs, SaaS vendors) need access to your account, the External ID prevents the "confused deputy" attack—a scenario where an unauthorized party tricks the third party into accessing your resources by using their ARN. The External ID acts as a secret shared only between you and the trusted third party.

**Cross-Account Access Automation:**

```bash
#!/bin/bash
# Script for engineers to assume cross-account role with MFA

CENTRAL_ACCOUNT_ID="123456789012"
TARGET_ACCOUNT_ID="210987654321"
ROLE_NAME="DeveloperAccess"
MFA_DEVICE_ARN="arn:aws:iam::${CENTRAL_ACCOUNT_ID}:mfa/john.doe"
DURATION=3600  # 1 hour

read -s -p "Enter MFA code: " MFA_CODE

# Assume role in central account (if using identity account pattern)
CREDS=$(aws sts assume-role \
    --role-arn "arn:aws:iam::${TARGET_ACCOUNT_ID}:role/${ROLE_NAME}" \
    --role-session-name "dev-session-$(date +%s)" \
    --serial-number "${MFA_DEVICE_ARN}" \
    --token-code "${MFA_CODE}" \
    --duration-seconds ${DURATION})

export AWS_ACCESS_KEY_ID=$(echo $CREDS | jq -r '.Credentials.AccessKeyId')
export AWS_SECRET_ACCESS_KEY=$(echo $CREDS | jq -r '.Credentials.SecretAccessKey')
export AWS_SESSION_TOKEN=$(echo $CREDS | jq -r '.Credentials.SessionToken')

echo "Temporary credentials exported. Valid for ${DURATION} seconds."
```

### 13.4.2 Azure AD B2B and AWS IAM Identity Center Integration

For organizations using Azure AD as their primary identity provider and AWS for infrastructure:

```json
// Azure AD Enterprise Application for AWS SSO
{
  "id": "aws-sso-app",
  "appRoles": [
    {
      "allowedMemberTypes": ["User"],
      "description": "Administrator access to AWS production",
      "displayName": "AWS-Production-Admin",
      "id": "guid-for-admin-role",
      "isEnabled": true,
      "value": "arn:aws:sso:::permissionSet/ssoins-1234567890123456/ps-1234567890abcdef0"
    },
    {
      "allowedMemberTypes": ["User"],
      "description": "Read-only access to AWS development",
      "displayName": "AWS-Dev-ReadOnly",
      "id": "guid-for-readonly-role",
      "isEnabled": true,
      "value": "arn:aws:sso:::permissionSet/ssoins-1234567890123456/ps-0987654321fedcba0"
    }
  ],
  "groupMembershipClaims": "ApplicationGroup",
  "optionalClaims": {
    "idToken": [
      {
        "name": "groups",
        "source": null,
        "essential": false,
        "additionalProperties": []
      }
    ]
  }
}
```

**Federation Flow:**
1. User authenticates to Azure AD (corporate SSO)
2. Azure AD SAML assertion maps to AWS IAM Identity Center
3. User selects AWS account and role from portal
4. AWS STS issues temporary credentials valid for that specific account/role combination

### 13.4.3 GCP Workload Identity Federation for Multi-Cloud

Accessing GCP resources from AWS or Azure without service account keys:

```hcl
# GCP Workload Identity Pool for AWS workloads
resource "google_iam_workload_identity_pool" "aws_pool" {
  workload_identity_pool_id = "aws-pool"
  display_name              = "AWS Workloads"
}

resource "google_iam_workload_identity_pool_provider" "aws_provider" {
  workload_identity_pool_id          = google_iam_workload_identity_pool.aws_pool.workload_identity_pool_id
  workload_identity_pool_provider_id = "aws-provider"
  
  attribute_mapping = {
    "google.subject"        = "assertion.arn"  # AWS IAM Role ARN
    "attribute.aws_account" = "assertion.account"
    "attribute.role"        = "assertion.arn.extract('assumed-role/{role}/')"
  }
  
  aws {
    account_id = "123456789012"
  }
}

# Allow specific AWS role to access GCP Secret Manager
resource "google_secret_manager_secret_iam_member" "aws_access" {
  project   = var.project_id
  secret_id = google_secret_manager_secret.api_key.secret_id
  role      = "roles/secretmanager.secretAccessor"
  member    = "principalSet://iam.googleapis.com/${google_iam_workload_identity_pool.aws_pool.name}/attribute.role/MyAppRole"
}
```

---

## 13.5 IAM Governance and Monitoring

Effective IAM requires continuous monitoring for policy violations, unused permissions, and anomalous access patterns.

### 13.5.1 Access Analyzer and Policy Validation

**AWS IAM Access Analyzer:**
Continuously analyzes resource policies to identify resources shared with external entities (outside your organization).

```python
# Lambda to remediate public S3 buckets discovered by Access Analyzer
import boto3
import json

def remediate_public_access(event, context):
    """
    Triggered by Access Analyzer finding of public S3 bucket
    """
    finding = json.loads(event['detail'])
    
    if finding['resourceType'] != 'AWS::S3::Bucket':
        return
    
    bucket_name = finding['resource']
    
    s3 = boto3.client('s3')
    
    # Immediate remediation: Remove public access
    try:
        s3.put_public_access_block(
            Bucket=bucket_name,
            PublicAccessBlockConfiguration={
                'BlockPublicAcls': True,
                'IgnorePublicAcls': True,
                'BlockPublicPolicy': True,
                'RestrictPublicBuckets': True
            }
        )
        
        # Notify security team
        sns = boto3.client('sns')
        sns.publish(
            TopicArn='arn:aws:sns:us-east-1:123456789012:security-alerts',
            Subject=f'Auto-remediated public S3 bucket: {bucket_name}',
            Message=json.dumps(finding)
        )
        
    except Exception as e:
        print(f"Failed to remediate {bucket_name}: {e}")
```

### 13.5.2 Permissions Boundaries vs. Service Control Policies

**Permissions Boundaries (Identity-Level):**
Maximum permissions an IAM entity can have, regardless of identity-based policies. Used to prevent privilege escalation even if a user has `iam:AttachUserPolicy` permissions.

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "BoundaryForDeveloperRole",
      "Effect": "Allow",
      "Action": [
        "ec2:*",
        "s3:GetObject",
        "s3:PutObject",
        "cloudwatch:*",
        "logs:*"
      ],
      "Resource": "*",
      "Condition": {
        "StringEquals": {
          "aws:RequestedRegion": ["us-east-1", "eu-west-1"]
        }
      }
    },
    {
      "Sid": "DenyDangerousActions",
      "Effect": "Deny",
      "Action": [
        "iam:*",
        "organizations:*",
        "account:*",
        "sts:AssumeRole"
      ],
      "Resource": "*"
    }
  ]
}
```

**Service Control Policies (Organization-Level):**
Guardrails applied across all accounts in an AWS Organization. Evaluated before identity-based policies.

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "RestrictRegions",
      "Effect": "Deny",
      "Action": "*",
      "Resource": "*",
      "Condition": {
        "StringNotEquals": {
          "aws:RequestedRegion": ["us-east-1", "us-west-2", "eu-west-1"]
        },
        "ArnNotLike": {
          "aws:PrincipalARN": ["arn:aws:iam::*:role/OrganizationAccountAccessRole"]
        }
      }
    },
    {
      "Sid": "PreventAccountLeaving",
      "Effect": "Deny",
      "Action": [
        "organizations:LeaveOrganization",
        "account:CloseAccount",
        "account:DeleteAlternateContact"
      ],
      "Resource": "*"
    }
  ]
}
```

### 13.5.3 Access Reviews and Recertification

**Automated Access Review Workflow:**

```python
# Quarterly access review automation
def generate_access_report(event, context):
    """
    Generates report of all IAM users and their last activity
    Triggers removal workflow for unused credentials
    """
    iam = boto3.client('iam')
    cloudtrail = boto3.client('cloudtrail')
    
    users = iam.list_users()['Users']
    
    for user in users:
        user_name = user['UserName']
        
        # Check last console login
        if 'PasswordLastUsed' in user:
            days_since_login = (datetime.now() - user['PasswordLastUsed']).days
            if days_since_login > 90:
                flag_for_review(user_name, 'Console', days_since_login)
        
        # Check access keys
        keys = iam.list_access_keys(UserName=user_name)['AccessKeyMetadata']
        for key in keys:
            last_used = iam.get_access_key_last_used(AccessKeyId=key['AccessKeyId'])
            if 'LastUsedDate' not in last_used['AccessKeyLastUsed']:
                # Key never used - immediate revocation candidate
                schedule_key_deletion(user_name, key['AccessKeyId'])
```

---

## 13.6 Chapter Summary and Transition

This chapter has established comprehensive Identity and Access Management as the cornerstone of cloud security, implementing architectures that transcend simple user management to embrace sophisticated, dynamic authorization models. We explored the evolution from static Role-Based Access Control to dynamic Attribute-Based Access Control (ABAC), enabling fine-grained permissions that automatically adjust based on organizational context, resource tags, and environmental conditions.

The elimination of long-term credentials through Workload Identity Federation represents a paradigm shift in service account security—replacing persistent secrets with ephemeral, cryptographically-bound tokens that eliminate the risk of credential leakage and reduce operational burden. We implemented Just-in-Time (JIT) access patterns that enforce true least privilege, granting elevated permissions only for specific tasks, with mandatory approval workflows, time-bound sessions, and automatic revocation.

Cross-account and cross-cloud access patterns demonstrated how to maintain security boundaries while enabling operational efficiency across distributed environments, utilizing External IDs to prevent confused deputy attacks and federation protocols to maintain single sources of truth for identity. Finally, we established governance frameworks combining preventative controls (Service Control Policies, Permissions Boundaries) with detective controls (Access Analyzer, automated remediation) to maintain continuous compliance.

However, even the most sophisticated IAM architecture cannot secure infrastructure that is fundamentally misconfigured. While identity controls govern *who* can access resources, the resources themselves must be hardened against attack—network security groups must restrict traffic, encryption must be enforced, logging must be comprehensive, and vulnerabilities must be remediated. IAM is the gatekeeper, but the infrastructure behind those gates must be architecturally secure.

In **Chapter 14: Securing Cloud Infrastructure**, we will shift focus from the identity layer to the resource layer. You will learn to implement defense-in-depth network architectures utilizing Web Application Firewalls, DDoS protection, and micro-segmentation; harden compute resources through secure configurations and vulnerability management; protect data through encryption key lifecycle management and secrets rotation; and establish comprehensive logging and monitoring infrastructures that provide visibility into security events across distributed cloud environments. We will explore the technical implementation of the security controls promised by the Shared Responsibility Model, transforming architectural principles into operational reality.